In [27]:
# outputs_path = "/home/zcwang/TRACE/outputs/cl/EPI/TinyLlama/TinyLlama-1.1B-Chat-v1.0/predictions"
outputs_path = "/home/zcwang/TRACE/outputs/cl/EPI/google/gemma-2b-it/predictions"

dataset_list = "C-STANCE,FOMC,Py150,ScienceQA,NumGLUE-cm,NumGLUE-ds,20Minuten".split(",")
metric_list = "accuracy,accuracy,similarity,accuracy,accuracy,accuracy,sari".split(",")
scale_list = [100, 100, 1, 100, 100, 100, 1]

import os
import json

task_count = len(dataset_list)
score_matrix = [[] for _ in range(task_count)]
for t in range(task_count):
    for i in range(t+1):
        file_path = os.path.join(outputs_path, f"results-{t}-{i}-{dataset_list[i]}.json")
        data = json.load(open(file_path))

        if "Qwen" in outputs_path:
            if dataset_list[i] == "FOMC":
                results = data["results"]
                labels = data["labels"]
                results = [res.strip() for res in results]
                # print(results, labels)
                acc = sum([1 for res, label in zip(results, labels) if res == label]) / len(labels)
                # print(f"Accuracy for {file_path}: {acc:.2f}")
                score_matrix[i].append(acc)
                continue

            elif dataset_list[i] == "ScienceQA":
                results = data["results"]
                labels = data["labels"]
                results = [res.strip() for res in results]
                labels = [label[0] for label in labels]
                results = [res[0] for res in results]
                # print(results, labels)
                acc = sum([1 for res, label in zip(results, labels) if res == label]) / len(labels)
                # print(f"Accuracy for {file_path}: {acc:.2f}")
                score_matrix[i].append(acc)
                continue

        if metric_list[i] != "sari":
            score_matrix[i].append(data["eval"][metric_list[i]])
        else:
            score_matrix[i].append(data["eval"]["sari"][0]["sari"])

# display the score matrix
print("Score matrix:")
for idx, row in enumerate(score_matrix):
    # expand the row to the full length
    row = ["-"] * (task_count - len(row)) + row + [row[-1] - row[0]]
    print(dataset_list[idx] + ' & ' + ' & '.join([f"{score*scale_list[idx]:.2f}" if not isinstance(score, str) else score for score in row]) + " \\\\")

Score matrix:
C-STANCE & 55.00 & 55.00 & 56.00 & 55.00 & 54.00 & 55.00 & 52.00 & -3.00 \\
FOMC & - & 57.00 & 54.00 & 53.00 & 53.00 & 54.00 & 54.00 & -3.00 \\
Py150 & - & - & 56.97 & 57.39 & 56.66 & 56.89 & 56.63 & -0.34 \\
ScienceQA & - & - & - & 46.00 & 48.00 & 46.00 & 46.00 & 0.00 \\
NumGLUE-cm & - & - & - & - & 34.57 & 35.80 & 32.10 & -2.47 \\
NumGLUE-ds & - & - & - & - & - & 48.00 & 46.00 & -2.00 \\
20Minuten & - & - & - & - & - & - & 39.60 & 0.00 \\
